# Importação de bibliotecas

In [3]:
import pandas as pd
import random
import locale
import os
import openai

# Declaração de variáveis

In [4]:
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

openai.api_key = os.getenv('OPENAI_API_KEY')

NUM_CARTAS_TOTAIS = 78
NUM_CARTAS_ESCOLHIDAS = 3
ARQUIVO_CARTAS = "C:/Users/PC/Desktop/Projetos/Tarot_Python/TAROT - TODAS.xlsx"

# Funções

In [5]:
def escolher_ids_aleatorios():
    return random.sample(range(1, NUM_CARTAS_TOTAIS + 1), NUM_CARTAS_ESCOLHIDAS)


In [6]:
def ler_arquivo(ids_escolhidos):
    df = pd.read_excel(ARQUIVO_CARTAS)
    cartas_selecionadas = df[df['ID'].isin(ids_escolhidos)]
    return cartas_selecionadas


In [7]:
def mostrar_cartas_selecionadas(cartas, conselho):
    for _, carta in cartas.iterrows():
        print(f"Nome: {carta['Carta']}")  
        print(f"Elemento: {carta['Elemento']}")
        print(f"Planeta: {carta['Planeta']}")
        print(f"Signo Astrológico: {carta['Signo_Astrologico']}")
        print(f"Kabbalah: {carta['Cabala_Kabbalah']}")
        print(f"Sim ou Não: {carta['Sim_Ou_Nao']}")
        print(f"Significado: {carta['Significado_Normal']}\n\n")
    print(f"Conselho: {conselho}\n\n")


In [8]:
def exibir_menu_e_capturar_escolha():
    temas = {
        1: "Amor",
        2: "Dinheiro",
        3: "Amigos",
        4: "Carreira",
        5: "Estudos",
        6: "Espiritualidade"
    }

    print("Pense em uma pergunta sobre os temas:")
    for chave, valor in temas.items():
        print(f"{chave} - {valor}")
    print("0 - Para sair")

    try:
        opcao = int(input("Escolha uma opção: "))
        if opcao == 0:
            return 0, "sair"
        elif opcao in temas:
            return opcao, temas[opcao]
        else:
            print("Opção inválida. Por favor, digite um número válido.")
            return None, None
    except ValueError:
        print("Por favor, digite um número válido.")
        return None, None


In [9]:
def gerar_conselho(tema, cartas_selecionadas):
    # Preparar o prompt com base nas cartas selecionadas
    cartas_info = ""
    for _, carta in cartas_selecionadas.iterrows():
        cartas_info += f"{carta['Carta']}: {carta['Significado_Normal']} ({'Sim' if carta['Sim_Ou_Nao'] == 'Sim' else 'Não'}).\n"

    prompt = f"Dê um conselho curto (máximo de 280 caracteres) sobre o tema {tema}, considerando as cartas: {', '.join(cartas_selecionadas['Carta'])}. Informações das cartas: {cartas_info}"

    try:
        #Chama a API
        resposta = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "Responda em português: Você é um tarólogo dando conselhos,..."},
                {"role": "user", "content": prompt + " " + cartas_info}
            ]
        )
        #resposta = 'chama'
        # Acessar a resposta gerada corretamente
        conselho = resposta.choices[0].message['content'].strip()
        return conselho  # Retorna o conselho aqui, dentro do try
    except Exception as e:
        conselho = f"Não foi possível gerar um conselho devido ao seguinte erro: {e}"
        # Aqui, também retorna o conselho em caso de erro, para garantir que a função nunca retorne None
        return conselho
    # Salvar o conselho em um arquivo
    with open("conselho.txt", "w", encoding="utf-8") as arquivo:
        arquivo.write(conselho)  

# Outra alteração é na função main, onde você deve verificar se conselho não é None antes de tentar usá-lo


In [10]:
def main():
    while True:
        opcao, tema_escolhido = exibir_menu_e_capturar_escolha()
        
        if opcao == 0:
            print("Volte sempre!")
            break
        elif opcao in [1, 2, 3, 4, 5, 6]:
            ids_escolhidos = escolher_ids_aleatorios()
            cartas_escolhidas = ler_arquivo(ids_escolhidos)
            conselho = gerar_conselho(tema_escolhido, cartas_escolhidas)  # Ajuste conforme necessário
            
            mostrar_cartas_selecionadas(cartas_escolhidas, conselho)
        else:
            continue
            print('deu ruim')


In [11]:
main()

Pense em uma pergunta sobre os temas:
1 - Amor
2 - Dinheiro
3 - Amigos
4 - Carreira
5 - Estudos
6 - Espiritualidade
0 - Para sair
Por favor, digite um número válido.
Pense em uma pergunta sobre os temas:
1 - Amor
2 - Dinheiro
3 - Amigos
4 - Carreira
5 - Estudos
6 - Espiritualidade
0 - Para sair
Nome: A Lua
Elemento: Agua
Planeta: Netuno
Signo Astrológico: Peixes
Kabbalah: Fevereiro e Marco
Sim ou Não: Nao
Significado: Ilusao, ansiedade, subconsciente, intuicao.


Nome: 6 de Ouros
Elemento: Terra
Planeta: Lua
Signo Astrológico: Touro
Kabbalah: Tipharet – Beleza
Sim ou Não: Sim
Significado: generosidade, equilibrio em dar e receber


Nome: Rei de Ouros
Elemento: Terra
Planeta: Mercurio
Signo Astrológico: Capricornio
Kabbalah: Chokmah – Sabedoria
Sim ou Não: Sim
Significado: sucesso, aperfeicoamento, seguranca, alto status, estabilidade, lealdade, conservador, gentil, empresario, confiavel


Conselho: No amor, tenha cuidado com ilusões e ansiedades que possam surgir. Lembre-se da importân